In [317]:
import pandas as pd

### [K-IFRS] 연결기업 - 별도기업 병합(영업이익, 당기순이익, 매출액)

In [360]:
# 데이터 불러오기
con = pd.read_csv('데이터/data/연결기업_성장률.csv', encoding='CP949')
sep = pd.read_csv('데이터/data/별도기업_성장률.csv', encoding='CP949')

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [361]:
# 데이터 프레임 행렬 수 출력

con.shape
## (20929, 6)

sep.shape
## (20929, 6)

(20929, 6)

In [363]:
# 회사 수

con['회사명'].nunique()
## 2454

sep['회사명'].nunique()
## 2454

2454

In [365]:
# 데이터 프레임 결측치 합 확인
con.isna().sum()

회사명                             0
거래소코드                           0
회계년도                            0
매출액(수익)(*)(IFRS연결)           6310
* (정상)영업손익(보고서기재)(IFRS연결)    6310
당기순이익(손실)(IFRS연결)            6310
dtype: int64

In [366]:
sep.isna().sum()

회사명                           0
거래소코드                         0
회계년도                          0
매출액(수익)(*)(IFRS)           1814
당기순이익(손실)(IFRS)            1814
* (정상)영업손익(보고서기재)(IFRS)    1814
dtype: int64

In [383]:
# 컬럼명 확인
con.columns

Index(['회사명', '거래소코드', '회계년도', '매출액', ' 영업손익', '당기순이익'], dtype='object')

In [384]:
# 컬럼명 불필요 문자 제거
con.columns = con.columns.str.replace('수익', '')
con.columns = con.columns.str.replace('(', '')
con.columns = con.columns.str.replace(')', '')
con.columns = con.columns.str.replace('IFRS연결', '')
con.columns = con.columns.str.replace('*', '')
con.columns = con.columns.str.replace('보고서기재', '')
con.columns = con.columns.str.replace('손실', '')
con.columns = con.columns.str.replace('정상', '')
con.columns = con.columns.str.replace(' ', '')
con.columns

<ipython-input-384-638e5e05a2ec>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace('(', '')
<ipython-input-384-638e5e05a2ec>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace(')', '')
<ipython-input-384-638e5e05a2ec>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '매출액', '영업손익', '당기순이익'], dtype='object')

In [385]:
# 컬럼명 확인
sep.columns

Index(['회사명', '거래소코드', '회계년도', '매출액', '당기순이익', '영업손익'], dtype='object')

In [386]:
# 컬럼명 불필요 문자 제거
sep.columns = sep.columns.str.replace('수익', '')
sep.columns = sep.columns.str.replace('(', '')
sep.columns = sep.columns.str.replace(')', '')
sep.columns = sep.columns.str.replace('IFRS', '')
sep.columns = sep.columns.str.replace('*', '')
sep.columns = sep.columns.str.replace('보고서기재', '')
sep.columns = sep.columns.str.replace('손실', '')
sep.columns = sep.columns.str.replace('정상', '')
sep.columns = sep.columns.str.replace(' ', '')
sep.columns

<ipython-input-386-7f5a5ff74380>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sep.columns = sep.columns.str.replace('(', '')
<ipython-input-386-7f5a5ff74380>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sep.columns = sep.columns.str.replace(')', '')
<ipython-input-386-7f5a5ff74380>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sep.columns = sep.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '매출액', '당기순이익', '영업손익'], dtype='object')

In [388]:
# 컬럼명 재정렬
con = con[['회사명', '거래소코드', '회계년도', '영업손익', '당기순이익', '매출액']]
# con.columns

Index(['회사명', '거래소코드', '회계년도', '영업손익', '당기순이익', '매출액'], dtype='object')

In [391]:
sep = sep[['회사명', '거래소코드', '회계년도', '영업손익', '당기순이익', '매출액']]
# sep.columns

In [395]:
# K-IFRS_연결 데이터프레임의 컬럼명을 리스트로 저장
con_list = con.columns.tolist()
print(con_list)

['회사명', '거래소코드', '회계년도', '영업손익', '당기순이익', '매출액']


In [396]:
# K-IFRS_연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in con_list:
    con[i] = con[i].fillna(sep[i])
con

,회사명,거래소코드,회계년도,영업손익,당기순이익,매출액
0,(주)CMG제약,58820,2011/12,640283.0,-1508282.0,17751704.0
1,(주)CMG제약,58820,2012/12,-4205503.0,-6660149.0,16255389.0
2,(주)CMG제약,58820,2013/12,791250.0,296181.0,20402140.0
3,(주)CMG제약,58820,2014/12,442322.0,158039.0,22752584.0
4,(주)CMG제약,58820,2015/12,-2572168.0,-3650091.0,27041894.0
...,...,...,...,...,...,...
20924,흥아해운(주),3280,2016/12,5895873.0,-17173253.0,831746081.0
20925,흥아해운(주),3280,2017/12,-13098624.0,-73500411.0,836427496.0
20926,흥아해운(주),3280,2018/12,-37595967.0,-86540156.0,753865569.0
20927,흥아해운(주),3280,2019/12,-12364293.0,-51356551.0,102166838.0


In [397]:
# 데이터 프레임 저장
con.to_csv('데이터/1차 전처리/K-IFRS_성장률_별도병합.csv', encoding='CP949')

In [398]:
# 데이터 프레임 결측치 합 확인
con.isna().sum()

회사명         0
거래소코드       0
회계년도        0
영업손익     1699
당기순이익    1699
매출액      1699
dtype: int64

--------

### [K-GAAP] 연결기업 - 별도기업 병합(영업이익, 당기순이익, 매출액)

In [318]:
# 데이터 불러오기
kgaap_con = pd.read_csv('데이터/data/K-GAAP_연결_성장률.csv', encoding='CP949')
kgaap_sep = pd.read_csv('데이터/data/K-GAAP_별도_성장률.csv', encoding='CP949')

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [319]:
# 데이터 프레임 행렬 수 출력

kgaap_con.shape
## (15706, 6)

kgaap_sep.shape
## (15706, 6)

(15706, 6)

In [320]:
# 회사 수

kgaap_con['회사명'].nunique()
## 1983

kgaap_sep['회사명'].nunique()
## 1983

1983

In [321]:
# 데이터 프레임 결측치 합 확인
kgaap_con.isna().sum()

회사명                    0
거래소코드                  0
회계년도                   0
매출액(영업수익)(*)(연결)    9626
영업이익(손실)(연결)        9626
당기순이익(순손실)(연결)      9626
dtype: int64

In [322]:
kgaap_sep.isna().sum()

회사명              0
거래소코드            0
회계년도             0
당기순이익(순손실)      75
매출액(영업수익)(*)    75
영업이익(손실)        75
dtype: int64

In [323]:
# 컬럼명 확인
kgaap_con.columns

Index(['회사명', '거래소코드', '회계년도', '매출액(영업수익)(*)(연결)', '영업이익(손실)(연결)',
       '당기순이익(순손실)(연결)'],
      dtype='object')

In [324]:
# 컬럼명 불필요 문자 제거
kgaap_con.columns = kgaap_con.columns.str.replace('순손실', '')
kgaap_con.columns = kgaap_con.columns.str.replace('(', '')
kgaap_con.columns = kgaap_con.columns.str.replace(')', '')
kgaap_con.columns = kgaap_con.columns.str.replace('영업수익', '')
kgaap_con.columns = kgaap_con.columns.str.replace('*', '')
kgaap_con.columns = kgaap_con.columns.str.replace('연결', '')
kgaap_con.columns = kgaap_con.columns.str.replace('손실', '')
kgaap_con.columns

<ipython-input-324-1eddc8a098f4>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_con.columns = kgaap_con.columns.str.replace('(', '')
<ipython-input-324-1eddc8a098f4>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_con.columns = kgaap_con.columns.str.replace(')', '')
<ipython-input-324-1eddc8a098f4>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_con.columns = kgaap_con.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '매출액', '영업이익', '당기순이익'], dtype='object')

In [325]:
# 컬럼명 확인
kgaap_sep.columns

Index(['회사명', '거래소코드', '회계년도', '당기순이익(순손실)', '매출액(영업수익)(*)', '영업이익(손실)'], dtype='object')

In [326]:
kgaap_sep.columns = kgaap_sep.columns.str.replace('순손실', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace('(', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace(')', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace('영업수익', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace('*', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace('연결', '')
kgaap_sep.columns = kgaap_sep.columns.str.replace('손실', '')
kgaap_sep.columns

<ipython-input-326-f7f998f6c04f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_sep.columns = kgaap_sep.columns.str.replace('(', '')
<ipython-input-326-f7f998f6c04f>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_sep.columns = kgaap_sep.columns.str.replace(')', '')
<ipython-input-326-f7f998f6c04f>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgaap_sep.columns = kgaap_sep.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '당기순이익', '매출액', '영업이익'], dtype='object')

In [327]:
# 컬럼명 재정렬
kgaap_con = kgaap_con[['회사명', '거래소코드', '회계년도', '영업이익', '당기순이익', '매출액']]
# kgaap_con.columns

In [328]:
kgaap_sep = kgaap_sep[['회사명', '거래소코드', '회계년도', '영업이익', '당기순이익', '매출액']]
# kgaap_sep.columns

In [329]:
# K-GAAP_연결 데이터프레임의 컬럼명을 리스트로 저장
kgaap_con_list = kgaap_con.columns.tolist()
print(kgaap_con_list)

['회사명', '거래소코드', '회계년도', '영업이익', '당기순이익', '매출액']


In [330]:
# K-GAAP_연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in kgaap_con_list:
    kgaap_con[i] = kgaap_con[i].fillna(kgaap_sep[i])
kgaap_con

,회사명,거래소코드,회계년도,영업이익,당기순이익,매출액
0,(주)CMG제약,58820,2001/12,294030.0,110431.0,15822816.0
1,(주)CMG제약,58820,2002/12,-3817019.0,-2254749.0,26394126.0
2,(주)CMG제약,58820,2003/12,-914055.0,110249.0,17389284.0
3,(주)CMG제약,58820,2004/12,-2838322.0,-3873234.0,13083977.0
4,(주)CMG제약,58820,2005/12,-1994985.0,-2558027.0,4186236.0
...,...,...,...,...,...,...
15701,흥아해운(주),3280,2006/12,-2655684.0,-290315.0,643616434.0
15702,흥아해운(주),3280,2007/12,10492202.0,-3587607.0,665589779.0
15703,흥아해운(주),3280,2008/12,11948162.0,770845.0,850131061.0
15704,흥아해운(주),3280,2009/12,-5586486.0,-17040555.0,671257274.0


In [331]:
# 데이터 프레임 저장
kgaap_con.to_csv('데이터/1차 전처리/K-GAAP_성장률_별도병합.csv', encoding='CP949')

In [332]:
# 데이터 프레임 결측치 합 확인
kgaap_con.isna().sum()

회사명       0
거래소코드     0
회계년도      0
영업이익     36
당기순이익    36
매출액      36
dtype: int64

--------

### 연결기업 - 별도기업 병합(자산, 자본)

In [399]:
# 데이터 불러오기
AC_con = pd.read_csv('데이터/data/자산자본_연결.csv', encoding='CP949')
AC_sep = pd.read_csv('데이터/data/자산자본_별도.csv', encoding='CP949')

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [402]:
# 데이터 프레임 행렬 수 출력
AC_con.shape
## (20929, 5)

AC_sep.shape
## (20929, 5)

(20929, 5)

In [404]:
# 회사 수
AC_con['회사명'].nunique()
## 2454

AC_sep['회사명'].nunique()
## 2454

2454

In [405]:
# 데이터 프레임 결측치 합 확인
AC_con.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
자산(*)(IFRS연결)    6310
자본(*)(IFRS연결)    6310
dtype: int64

In [406]:
AC_sep.isna().sum()

회사명               0
거래소코드             0
회계년도              0
자산(*)(IFRS)    1814
자본(*)(IFRS)    1814
dtype: int64

In [407]:
# 컬럼명 확인
AC_con.columns

Index(['회사명', '거래소코드', '회계년도', '자산(*)(IFRS연결)', '자본(*)(IFRS연결)'], dtype='object')

In [408]:
# 컬럼명 불필요 문자 제거
AC_con.columns = AC_con.columns.str.replace('IFRS', '')
AC_con.columns = AC_con.columns.str.replace('(', '')
AC_con.columns = AC_con.columns.str.replace(')', '')
AC_con.columns = AC_con.columns.str.replace('영업수익', '')
AC_con.columns = AC_con.columns.str.replace('*', '')
AC_con.columns = AC_con.columns.str.replace('연결', '')
AC_con.columns

<ipython-input-408-04d226cfdbd5>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_con.columns = AC_con.columns.str.replace('(', '')
<ipython-input-408-04d226cfdbd5>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_con.columns = AC_con.columns.str.replace(')', '')
<ipython-input-408-04d226cfdbd5>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_con.columns = AC_con.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '자산', '자본'], dtype='object')

In [409]:
AC_sep.columns

Index(['회사명', '거래소코드', '회계년도', '자산(*)(IFRS)', '자본(*)(IFRS)'], dtype='object')

In [410]:
AC_sep.columns = AC_sep.columns.str.replace('IFRS', '')
AC_sep.columns = AC_sep.columns.str.replace('(', '')
AC_sep.columns = AC_sep.columns.str.replace(')', '')
AC_sep.columns = AC_sep.columns.str.replace('영업수익', '')
AC_sep.columns = AC_sep.columns.str.replace('*', '')
AC_sep.columns = AC_sep.columns.str.replace('연결', '')
AC_sep.columns

<ipython-input-410-ff08e0d0b970>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_sep.columns = AC_sep.columns.str.replace('(', '')
<ipython-input-410-ff08e0d0b970>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_sep.columns = AC_sep.columns.str.replace(')', '')
<ipython-input-410-ff08e0d0b970>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  AC_sep.columns = AC_sep.columns.str.replace('*', '')


Index(['회사명', '거래소코드', '회계년도', '자산', '자본'], dtype='object')

In [411]:
# 자산자본_연결 데이터프레임의 컬럼명을 리스트로 저장
AC_con_list = AC_con.columns.tolist()
print(AC_con_list)

['회사명', '거래소코드', '회계년도', '자산', '자본']


In [412]:
# 연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in AC_con_list:
    AC_con[i] = AC_con[i].fillna(AC_sep[i])
AC_con

,회사명,거래소코드,회계년도,자산,자본
0,(주)CMG제약,58820,2011/12,27305198.0,18034249.0
1,(주)CMG제약,58820,2012/12,57108035.0,31590685.0
2,(주)CMG제약,58820,2013/12,55912563.0,48997281.0
3,(주)CMG제약,58820,2014/12,58068326.0,49002644.0
4,(주)CMG제약,58820,2015/12,56532821.0,46806888.0
...,...,...,...,...,...
20924,흥아해운(주),3280,2016/12,958532259.0,192487245.0
20925,흥아해운(주),3280,2017/12,848958520.0,116884496.0
20926,흥아해운(주),3280,2018/12,803698457.0,73148175.0
20927,흥아해운(주),3280,2019/12,430962722.0,30919383.0


In [413]:
# 데이터 프레임 저장
AC_con.to_csv('데이터/1차 전처리/자산자본_별도병합.csv', encoding='CP949')

In [414]:
# 데이터 프레임 결측치 합 확인
AC_con.isna().sum()

회사명         0
거래소코드       0
회계년도        0
자산       1696
자본       1696
dtype: int64

----

### 총자산증가율, 자기자본증가율 파생변수 구하기

In [415]:
AC_con2 = AC_con.copy()
AC_con2 = AC_con2.dropna(axis=0)

In [416]:
AC_con2['회사명'].nunique()

2390

In [417]:
AC_con2['전기자산'] = AC_con2['자산'].shift().where(AC_con2.회사명.eq(AC_con2.회사명.shift()))
AC_con2['전기자본'] = AC_con2['자본'].shift().where(AC_con2.회사명.eq(AC_con2.회사명.shift()))
# AC_con2

In [418]:
AC_con2['총자산증가율'] = ((AC_con2['자산'] - AC_con2['전기자산'])/AC_con2['전기자산'])*100
AC_con2['자기자본증가율'] = ((AC_con2['자본'] - AC_con2['전기자본'])/abs(AC_con2['전기자본']))*100
# AC_con2

In [419]:
AC_con2.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '전기자산', '전기자본', '총자산증가율',
       '자기자본증가율'],
      dtype='object')

In [420]:
# 컬럼명 재정렬
AC_con2 = AC_con2[['회사명', '거래소코드', '회계년도', '자산', '전기자산', '총자산증가율', '자본', '전기자본', '자기자본증가율']]
# AC_con2

In [421]:
AC_con2

,회사명,거래소코드,회계년도,자산,전기자산,총자산증가율,자본,전기자본,자기자본증가율
0,(주)CMG제약,58820,2011/12,27305198.0,NaN,NaN,18034249.0,NaN,NaN
1,(주)CMG제약,58820,2012/12,57108035.0,27305198.0,109.147119,31590685.0,18034249.0,75.170505
2,(주)CMG제약,58820,2013/12,55912563.0,57108035.0,-2.093352,48997281.0,31590685.0,55.100407
3,(주)CMG제약,58820,2014/12,58068326.0,55912563.0,3.855597,49002644.0,48997281.0,0.010946
4,(주)CMG제약,58820,2015/12,56532821.0,58068326.0,-2.644307,46806888.0,49002644.0,-4.480893
...,...,...,...,...,...,...,...,...,...
20924,흥아해운(주),3280,2016/12,958532259.0,791640802.0,21.081715,192487245.0,173076634.0,11.215038
20925,흥아해운(주),3280,2017/12,848958520.0,958532259.0,-11.431409,116884496.0,192487245.0,-39.276758
20926,흥아해운(주),3280,2018/12,803698457.0,848958520.0,-5.331246,73148175.0,116884496.0,-37.418411
20927,흥아해운(주),3280,2019/12,430962722.0,803698457.0,-46.377560,30919383.0,73148175.0,-57.730479


In [422]:
# 데이터 프레임 결측치 합 확인 -> 기업 수 만큼 결측시 생김(shift후 생긴 공백)
AC_con2.isna().sum()

회사명           0
거래소코드         0
회계년도          0
자산            0
전기자산       2390
총자산증가율     2390
자본            0
전기자본       2390
자기자본증가율    2390
dtype: int64

In [423]:
AC_con2 = AC_con2.dropna(axis=0)

In [424]:
# 데이터 프레임 저장
AC_con2.to_csv('데이터/1차 전처리/총자산_자기자본증가율.csv', encoding='CP949')

-----

In [425]:
# 데이터 불러오기
emp = pd.read_csv('데이터/data/종업원수증가율.csv', encoding='CP949')

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [427]:
# 데이터 프레임 행렬 수 출력
emp.shape

(20929, 4)

In [429]:
# 회사 수
emp['회사명'].nunique()

2454

In [430]:
# 데이터 프레임 결측치 합 확인
emp.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
종업원수증가율(IFRS)    2058
dtype: int64

In [432]:
# 컬럼명 확인
emp.columns

Index(['회사명', '거래소코드', '회계년도', '종업원수증가율(IFRS)'], dtype='object')

In [433]:
# 컬럼명 불필요 문자 제거
emp.columns = emp.columns.str.replace('IFRS', '')
emp.columns = emp.columns.str.replace('(', '')
emp.columns = emp.columns.str.replace(')', '')
emp.columns

<ipython-input-433-8f9774c4687e>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  emp.columns = emp.columns.str.replace('(', '')
<ipython-input-433-8f9774c4687e>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  emp.columns = emp.columns.str.replace(')', '')


Index(['회사명', '거래소코드', '회계년도', '종업원수증가율'], dtype='object')

In [435]:
# 데이터 프레임 저장
emp.to_csv('데이터/1차 전처리/종업원수증가율_전처리.csv', encoding='CP949')

In [436]:
# 데이터 프레임 결측치 합 확인
emp.isna().sum()

회사명           0
거래소코드         0
회계년도          0
종업원수증가율    2058
dtype: int64